In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from rich import print

from langchain_core.embeddings import Embeddings
from typing import List
from langchain_postgres import PGVectorStore, PGEngine
from models.embedding_model import embedding_engine

load_dotenv()

USER = os.getenv("db_user")
PASSWORD = os.getenv("db_password")
HOST = os.getenv("db_host")
PORT = os.getenv("db_port")
DBNAME = os.getenv("db_name")


In [2]:
folder_path = "data"

pdf_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

all_pages = []  # To store pages from all files

for file_path in pdf_files:
    loader = PyMuPDFLoader(file_path=file_path)

    # Load the documents for this file and append to `all_pages`
    for doc in loader.lazy_load():
        all_pages.append(doc)

# Output all collected pages
print(f"Total pages loaded: {len(all_pages)}")

Total pages loaded: 17

In [3]:
# Splitting document
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=300,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
)

In [4]:
splits = text_splitter.split_documents(all_pages)
print(f"Total chunks created: {len(splits)}")

Total chunks created: 34

In [5]:
CONNECTION_STRING = f"postgresql+asyncpg://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}"
pg_engine = PGEngine.from_connection_string(url=CONNECTION_STRING)

In [6]:
# len(embedding_engine.embed_query("test"))  # Test embedding engine initialization

In [7]:
from sqlalchemy.exc import ProgrammingError

from langchain_postgres import Column

TABLE_NAME = "jp_ai_bot"
VECTOR_SIZE = 3072

try:
    await pg_engine.ainit_vectorstore_table(
        table_name=TABLE_NAME,
        vector_size=VECTOR_SIZE,
        # schema_name="japan",
        id_column=Column("langchain_id", "VARCHAR"),
        # metadata_columns=[
        #     Column("likes", "INTEGER"),
        #     Column("location", "TEXT"),
        #     Column("topic", "TEXT"),
        # ],
    )
except ProgrammingError:
    # Catching the exception here
    print("Table already exists. Skipping creation.")

Table already exists. Skipping creation.

In [8]:
async def create_vector_store(
    table_name: str, schema_name: str = "public"
) -> PGVectorStore:
    vector_store = await PGVectorStore.create(
        engine=pg_engine,
        schema_name=schema_name,
        embedding_service=embedding_engine,
        table_name=table_name,
    )
    print("PGVector Store is loaded.")
    return vector_store


vector_store = await create_vector_store(
    table_name="jp_ai_bot",
    # schema_name="japan",
)

PGVector Store is loaded.

In [9]:
# push embedding to collection
print("Adding documents to vector store...")
for i in range(0, len(splits), 3):
    chunk = splits[i : i + 3]
    try:
        # Add the chunk to the vector store
        await vector_store.aadd_documents(documents=chunk)
        print(f"Chunk {i // 3 + 1}/{(len(splits) + 3) // 3} added successfully")
    except Exception as e:
        print(f"Error adding chunk {i // 3}: {e}")
        continue

Adding documents to vector store...

Chunk 1/12 added successfully

Chunk 2/12 added successfully

Chunk 3/12 added successfully

Chunk 4/12 added successfully

...

Chunk 8/12 added successfully

Chunk 9/12 added successfully

Chunk 10/12 added successfully

Chunk 11/12 added successfully

Chunk 12/12 added successfully

In [12]:
query = "am i eligible for Subsidy Program"
docs = await vector_store.asimilarity_search(query)

for doc in docs:
    print(repr(doc))

Document(id='a48d2665-a668-4615-97ff-9d3a26d987b2', metadata={'producer': 'Microsoft® Word 2021', 'creator': 
'Microsoft® Word 2021', 'creationdate': '2024-02-09T15:04:51+09:00', 'source': 'data\\FAQs.pdf', 'file_path': 
'data\\FAQs.pdf', 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '中小機構', 'subject': '', 
'keywords': '', 'moddate': '2024-02-09T15:04:51+09:00', 'trapped': '', 'modDate': "D:20240209150451+09'00'", 
'creationDate': "D:20240209150451+09'00'", 'page': 0}, page_content='Q2-8 
以前、持続化補助金の卒業枠で採択を受けて、補助事業を実施したが、従業員\n数が減ったので今回の申請はできますか。 
\nQ2-９ 登記は日本で行ったが、事業拠点は海外にあり、海外に在住しています。申請は\nできますか。 
\n3．補助対象事業・経費 \n \nQ3-1 本店と支店がありますが、支店で補助事業を行う場合も対象となりますか。 \nP.４ 
\nQ3-2 支店で補助事業を行う場合、常時使用する従業員の数え方はどうなりますか。 \nQ3-3 
商品サンプル試供品製作は対象ですか。 \nQ3-4 ホームページ制作は対象となりますか。 \nQ3-5 
他の補助金との併用はできますか。 \nQ3-6 海外で実施する事業は対象となりますか。 \nQ3-7 
「汎用性があり目的外使用になるもの～」とは、具体的にどのようなものですか。 \nQ3-8 
公募要領に記載のないものを購入等する場合、補助対象か分かりません。 \nQ3-9 
ホームページ作成を業者に依頼する場合の経費区分はなんですか。')

Document(id='d0df3c79-e16f-4754-a7db-e4681fb656ba', metadata={'producer': 'Microsoft® Word 2021', 'creator': 
'Microsoft® Word 2021', 'creationdate': '2024-02-09T15:04:51+09:00', 'source': 'data\\FAQs.pdf', 'file_path': 
'data\\FAQs.pdf', 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '中小機構', 'subject': '', 
'keywords': '', 'moddate': '2024-02-09T15:04:51+09:00', 'trapped': '', 'modDate': "D:20240209150451+09'00'", 
'creationDate': "D:20240209150451+09'00'", 'page': 3}, page_content='P.12 \nQ6-2４ 
インボイス特例について補助事業終了時点において要件を満たさない場合、\n補助金の交付は行われますか。 
\n7．電子申請システムについて \n \nQ7-1  電子申請システムの登録方法等詳細の問い合わせ先はどこですか。 \nP.１２ 
\nQ7-2 
一時保存しているが、次回申請時に一時保存の情報を活用できますか。また、\n一時保存のまま締切を過ぎてしまった申請内容
は、別の回で参照することはで\nきますか。 \n8．その他 \n \nQ8-1 
補助事業終了後、補助金により購入したものを他者に売ったり貸したりすること\nはできますか。 \nP.1２ \nQ8-2 
パワーアップ型加点とはどのようなものでしょうか。自社の事業は加点対象とな\nりますか。 \nP.13 \nQ8-3 
政策加点審査とはどのようなことですか。 \nQ8-4 商工会・商工会議所の窓口の受付時間を教えてください。 \nQ8-５ 
賃金引上げ枠での申請もしくは賃上げ加点を受けた上で、本補助金で採択され\nました。賃金引上げの達成状況はいつ確認しま
すか。 \nQ8-６ 賃金引上げに係る要件が未達（計画通り賃金を引き上げていない）の場合どうな\nりますか。')

Document(id='b2fa8606-d662-426b-832b-92dc7a30ed9f', metadata={'producer': 'Microsoft® Word 2021', 'creator': 
'Microsoft® Word 2021', 'creationdate': '2024-02-09T15:04:51+09:00', 'source': 'data\\FAQs.pdf', 'file_path': 
'data\\FAQs.pdf', 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '中小機構', 'subject': '', 
'keywords': '', 'moddate': '2024-02-09T15:04:51+09:00', 'trapped': '', 'modDate': "D:20240209150451+09'00'", 
'creationDate': "D:20240209150451+09'00'", 'page': 0}, page_content='申請時によくあるご質問 
\n【第1５回公募以降対象】 \n「公募要領」、別紙「参考資料」を必ずご確認ください。 \n1．総論 \n \nQ1-1 
補助金が交付されるまでの流れを教えてください。 \nP.１ \nQ1-2 審査結果はいつわかりますか。 \nQ1-3 
提出した書類等が公開されることはありますか。 \nQ1-4 実績報告書（事業完了報告）の提出にあたり、何が必要ですか。 
\nQ1-5 商工会地区と商工会議所地区のどちらの窓口へ申請したら良いですか。 \nQ1-6 
商工会議所地区と商工会地区の窓口が２つありますが、双方に申請した場合は、\nどうなりますか。 \nP.2 \nQ1-7 
自社が商工会地区と商工会議所地区のどちらに該当しますか。 \nQ1-8 
事業支援計画書（様式４）発行の受付について、具体的な手続きはなんですか。 \nQ1-9 
補助金の採択・不採択について、会社名は公表されますか。 \nQ1-10 不採択となった場合、次回の公募に応募できますか。 
\nQ1-11 採択発表はどのようにされますか。 \n2．補助対象者について \n \nQ2-1 
商工会、商工会議所の会員以外でも、応募できますか。 \nP.2 \nQ2-2 士業を営んでいますが、補助の対象になりますか。 
\nQ2-3 派遣社員は「常時使用する従業員」に含まれますか。 \nQ2-4 これから開業する人は対象となりますか。 \nP.3 \nQ2-5 
申請期間中に個人事業主から法人に変更予定ですが、申請はできますか。 \nQ2-6 
屋号/法人が複数ありますが、どの屋号/法人で申請すればいいですか。 \nQ2-7 
以前、持続化補助金で採択を受けましたが、今回の申請はできますか。 \nQ2-8 
以前、持続化補助金の卒業枠で採択を受けて、補助事業を実施したが、従業員\n数が減ったので今回の申請はできますか。 
\nQ2-９ 登記は日本で行ったが、事業拠点は海外にあり、海外に在住しています。申請は\nできますか。 
\n3．補助対象事業・経費 \n \nQ3-1 本店と支店がありますが、支店で補助事業を行う場合も対象となりますか。 \nP.４ 
\nQ3-2 支店で補助事業を行う場合、常時使用する従業員の数え方はどうなりますか。 \nQ3-3 
商品サンプル試供品製作は対象ですか。 \nQ3-4 ホームページ制作は対象となりますか。 \nQ3-5 
他の補助金との併用はできますか。')

Document(id='45901473-a94f-4798-972a-8cfe096947ec', metadata={'producer': 'Microsoft® Word 2019', 'creator': 
'Microsoft® Word 2019', 'creationdate': '2024-02-07T11:52:19+09:00', 'source': 
'/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf', 'file_path': 
'/home/muntasirfahim/personal/JP_AI_Chatbot/data/demo.pdf', 'total_pages': 30, 'format': 'PDF 1.7', 'title': '', 
'author': '會田 裕登', 'subject': '', 'keywords': '', 'moddate': '2024-02-07T11:52:19+09:00', 'trapped': '', 
'modDate': "D:20240207115219+09'00'", 'creationDate': "D:20240207115219+09'00'", 'page': 7}, page_content='7 \n 
\n＜上記において「該当しない」を選択した事業者が対象＞ 
\n出資者の名称と出資比率および出資者の資本金を記載してく\nださい。（記載例：出資者の名称○○、出資者の資本金■■円、申
請\n者の資本金に占める出資比率▲▲％） \n※注・出資者が複数いる場合は、代表者１名分のみご記載ください。 
\n※注・株式会社・有限会社以外の法人の場合（合名会社、企業組合・協業組合\n等）は以下の通り記載ください。 
\n出資者の名称：該当しません、 出資者の資本金：0、 出資比率：0 \n出資者の名称（※） \n \n出資者の資本金（円）\n（※）
\n \n申請者の資本金に占\nめる出資比率（※）  \n \n＜全ての事業者が対象＞ \n過去３年のうち課税所得額が15 
億円超の年がある。 \n（課税所得が15 億円超の年がある場合は、過去３年分の課税所得\n額を記載してください。）※ 
\n注・上記への該当の有無の確認のため、必要がある場合には、納税証明書等の提出\nを求めることがあります。 
\n※確定している（申告済みの）直近過去３年分の「各年」又は「各事\n業年度」の課税所得の年平均額が15 
億円を超えている場合、申\n請はできません。 \n□該当する \n（過去3 年間の課税所得額\nを記載してください） 
\n□該当しない \n（いずれも15 億円以下） \n（前年\n）   億円 \n（2 年前）   億円 \n（3 年前）   億円 
\n＜全ての事業者が対象＞ 
\n補助対象事業として取り組むものが、「射幸心をそそるおそれがある、ま\nたは公序良俗を害するおそれがある」事業（公募
要領Ｐ．７参照）か否か。 \n□該当する \n（該当する場合は応\n募できません） \n□該当しない \n＜全ての事業者が対象＞ 
\n希望する枠いずれかを選択（チェック）してください（複数の類型に重複して申請することはできません）。 
\n※「賃金引上げ枠」を希望する赤字事業者のみ、2 つ（「賃金引上げ枠」と「赤字事業者」）にチェックを入れてください。 
\n \n希望する枠にチェック \n追加要件等 \n通常枠 \n□ \n― \n賃金引上げ枠 \n□ \n補助事業の終了時点において 
、事業場内最低賃金が申請時の地域別最低賃金\nより＋5０円以上であること（すでに事業場内最低賃金が地域別最低賃金より＋
\n5０円以上を達成している場合は、現在支給している、事業場内最低賃金より＋')

In [11]:
query_vector = embedding_engine.embed_query(query)
docs = await vector_store.asimilarity_search_by_vector(query_vector, k=2)
print(docs)

[
    Document(
        id='08f1bb0f-765c-48c4-b50c-5e8e1306bbeb',
        metadata={
            'producer': 'Microsoft: Print To PDF',
            'creator': '',
            'creationdate': '2024-03-28T21:33:11+09:00',
            'source': 'data\\Small Business Sustainability Subsidy (General Type) Grant Regulations.pdf',
            'file_path': 'data\\Small Business Sustainability Subsidy (General Type) Grant Regulations.pdf',
            'total_pages': 33,
            'format': 'PDF 1.7',
            'title': '',
            'author': '',
            'subject': '',
            'keywords': '',
            'moddate': '2024-06-05T11:57:57+09:00',
            'trapped': '',
            'modDate': "D:20240605115757+09'00'",
            'creationDate': "D:20240328213311+09'00'",
            'page': 9
        },
        page_content='様式および別紙一覧 \n \n \n様式第１   小規模事業者持続化補助金交付申請書 \n \n様式第２   
小規模事業者持続化補助金交付決定通知書 \n \n様式第３   小規模事業者持続化補助金交付申請取下届出書 \n \n様式第４   
小規模事業者持続化補助金に係る補助事業の内容・経費の配分の変更承認 \n申請書 \n \n別紙１   経費の配分の変更 \n 
\n様式第５   小規模事業者持続化補助金に係る補助事業の中止（廃止）申請書 \n \n様式第６   
小規模事業者持続化補助金に係る補助事業の事故報告書 \n \n様式第７   
小規模事業者持続化補助金に係る補助事業遂行状況報告書 \n \n 別紙２   支出内訳書 \n \n様式第８   
小規模事業者持続化補助金に係る補助事業実績報告書 \n \n 別紙３   支出内訳書 \n \n 別紙４   収益納付に係る報告書 \n 
\n 別紙５   賃金引上げ枠・賃上げ加点に係る実施報告書 \n \n様式第９   
小規模事業者持続化補助金に係る補助金精算払請求書 \n \n様式第10   消費税および地方消費税額の額の確定に伴う報告書 \n 
\n様式第11－１ 取得財産等管理台帳 \n \n様式第11－２ 取得財産等管理明細表 \n \n様式第12   取得財産の処分承認申請書 
\n \n様式第13   産業財産権等取得等届出書 \n \n様式第14   
小規模事業者持続化補助金に係る事業効果および賃金引上げ等状況報告 \n       書'
    ),
    Document(
        id='ab206f78-94e6-4bac-b906-31d3d9c541b2',
        metadata={
            'producer': 'Microsoft: Print To PDF',
            'creator': '',
            'creationdate': '2024-03-28T21:33:11+09:00',
            'source': 'data\\Small Business Sustainability Subsidy (General Type) Grant Regulations.pdf',
            'file_path': 'data\\Small Business Sustainability Subsidy (General Type) Grant Regulations.pdf',
            'total_pages': 33,
            'format': 'PDF 1.7',
            'title': '',
            'author': '',
            'subject': '',
            'keywords': '',
            'moddate': '2024-06-05T11:57:57+09:00',
            'trapped': '',
            'modDate': "D:20240605115757+09'00'",
            'creationDate': "D:20240328213311+09'00'",
            'page': 10
        },
        page_content='（様式第１） \n記入日：     年  月  日 \n全国商工会連合会 会長 殿 \n                     
郵便番号 \n         住  所 \n         名  称 \n             代表者の役職・氏名         印 \n               \n 
\n小規模事業者持続化補助金交付申請書 \n \n 
小規模事業者持続化補助金＜一般型＞交付規程第６条第１項の規定に基づき、上記補助\n金の交付について、下記のとおり申請
します。 \n（注）２、５、６のみ漏れなくご記入ください \n \n記 \n１．補助事業の目的および内容 \n   
補助事業計画書のとおり \n＊補助事業計画書は、全国連が指定する様式（公募要領様式）を使用すること。以下同様。 \n 
\n２．補助事業の開始日および完了予定日 \n   交付決定日 ～20●  年  月  日 \n \n３．補助対象経費 \n   
補助事業計画書のとおり \n \n４．補助金交付申請額 \n   補助事業計画書のとおり \n 
\n５．補助事業に関して生ずる収入金に関する事項（該当するものに○）  \n    （１）あり ／ （２）なし \n 
\n＊「（１）あり」の場合は以下に該当事項をご記入ください。 \n   該当事項：                      \n 
\n６．消費税の適用に関する事項（該当するもの一つに○） \n \n  （１）課税事業者 ／（２）免税事業者 
／（３）簡易課税事業者 ／（４）2 割特例（予定含む） 
\n＊消費税の適用区分によって、補助対象経費の算定方法が異なります。 \n＊２割特例が適用される対象者は、令和５年10 
月１日から令和８年９月30 日までの日の属する各課税期間に事業\nが完了する事業者となります。'
    )
]